In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 

import sklearn
from sklearn.cluster import KMeans
from collections import Counter

from tqdm import tqdm
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

from urllib.request import urlretrieve

import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://www.williamsburgoils.com/products/colors'
# prepare the option for the chrome driver
options = webdriver.ChromeOptions()
options.add_argument('headless')

# start chrome browser
browser = webdriver.Chrome(options=options)

# create action chain object
action = ActionChains(browser)

# open url
browser.get(url)

# allow to load
time.sleep(5)

In [3]:
colors = []

for e in tqdm(browser.find_elements(By.CLASS_NAME,"colorColumn")):
    color = {}
    text = e.text.split('\n')
    color['name'] = text[0]
    color['alt_name'] = text[1]
    for line in e.find_elements(By.TAG_NAME, 'img'):
        color['width'] = line.get_attribute('width')
        color['height'] = line.get_attribute('height')
        color['swatch_image_url'] = line.get_attribute('src')
    colors.append(color)
    time.sleep(2)

df = pd.DataFrame(colors)

100%|██████████| 176/176 [05:59<00:00,  2.04s/it]


In [4]:
df

,name,alt_name,width,height,swatch_image_url
0,Brilliant Yellow Extra Pale,#6000202,53,53,https://goldenhub.goldenpaints.com/storage/upl...
1,Brilliant Yellow Pale,#6000212,53,53,https://goldenhub.goldenpaints.com/storage/upl...
2,Nickel Yellow,#6000224,53,53,https://goldenhub.goldenpaints.com/storage/upl...
3,Bismuth Vanadate Yellow,#6001929,53,53,https://goldenhub.goldenpaints.com/storage/upl...
4,Cadmium Lemon,#6000246,53,53,https://goldenhub.goldenpaints.com/storage/upl...
...,...,...,...,...,...
171,Iridescent Pewter,#6001843,53,53,https://goldenhub.goldenpaints.com/storage/upl...
172,Interference Violet,#6001813,53,53,https://goldenhub.goldenpaints.com/storage/upl...
173,Interference Red,#6001873,53,53,https://goldenhub.goldenpaints.com/storage/upl...
174,Interference Blue,#6001803,53,53,https://goldenhub.goldenpaints.com/storage/upl...


In [9]:
df['path_to_image'] = df['name'].map(lambda x: f"./images/{x.lower().replace(' ','_')}_swatch.jpg")
df.to_csv('./williamsburg_oil_colors.csv', index = False)

In [8]:
for i in tqdm(range(len(df))):
    img_url = df.loc[i,'swatch_image_url']
    save_path = df.loc[i,'path_to_image']
    
    urlretrieve(img_url, save_path)
    time.sleep(3)

100%|██████████| 176/176 [08:59<00:00,  3.07s/it]
